# Tutorial 1: Training a pairwise G-P map on MPSA data.

In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# For showing images
from IPython.display import Image

# Insert path to mavenn beginning of path
import sys
sys.path.insert(0, '../../../')

# Load mavenn
import mavenn

In this tutorial we will train a G-P map on data from a massivley parallel splicing assay (MPSA) performed by Wong et al., 2018. Wong et al. mesured splicing activity for nearly all 5' splice sites of the form 'NNN/GYNNNN', where 'Y' denotes any pyrimidine ('C' or 'U'), and '/' denotes the exon/intron boundary.  This was accomplished using the experimental setup illustrated in the following figure. 

<img src="fig_tutorial1_fig1.png" alt="MPSA experiment" style="width: 600px;"/>

Specifically, the authors used a minigene expression system where each variant 5' splice site was tagged with a 20 nucleotide barcode. Constructs were transfected into cell culture, and the fraction of processed transcripts containing the central exon (which the variant 5' splice sites flank). The resulting dataset conisted of a list of assayed splice sites, each assigned a percent-spliced-in (PSI) value. Our goal is to model log PSI values as a function of sequence. 

We begin by loading data from this assay. This is provided as an example dataset within MAVE-NN, and can be loaded as follows.

In [2]:
# Load data as a pandas DataFrame
data_df = mavenn.load_example_dataset('mpsa')

# Show dataset size
print(f'Number of splice sites assayed: {len(data_df):,d}.')

# Preview dataset
data_df.head()

Number of splice sites assayed: 31,197.


,y,x
0,-0.427684,AAAGCAAAA
1,-0.162168,AAAGCAAAC
2,-0.121273,AAAGCAAAG
3,-0.353582,AAAGCAAAU
4,-0.260932,AAAGCAACA


In this dataset, $x$ denotes splice site sequences and $y$ denotes log PSI values (which the experiment determins up to an uncertain additive shift). We next extract the data from this dataframe and split into training and test sets. 

In [3]:
# Extract x and y as Numpy arrays
x = data_df['x'].values
y = data_df['y'].values

# Split data 80/20 into training / test sets. 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20)

# Show dataset sizes
print(f'Training set size: {len(x_train):6,d} observations')
print(f'Test set size    : {len(x_test):6,d} observations')

Training set size: 24,957 observations
Test set size    :  6,240 observations


Next we define a MAVE-NN model.


In [ ]:
# Define model and set training data
model = mavenn.Model(x=x_train, 
                     y=y_train,
                     alphabet='rna',
                     gpmap_type='pairwise',
                     regression_type='GE',
                     ge_noise_model_type='Gaussian',
                     ge_heteroskedasticity_order=2)
model

The overall form of this model is schematized in the following figure.

<img src="fig_tutorial1_fig2.png" alt="Modeling assumptions" style="width: 600px;"/>

In addition to specifying the training inputs (`x=x_train`) and training labels (`y=y_train`), five additional parameters are set. 

`alphabet='rna'`: This indicates that sequences are made up of the characters 'A', 'C', 'G', 'U'. 

`gpmap_type='pairwise'`: This indicates that we want to fit a pairwise model for the latent phenotype $\phi$, i.e. a model of the mathematical form

$$ \phi(x) = \theta_0 + \sum_{l,c} \theta_{l:c} x_{l:c} + \sum_{l,l'>l} \sum_{c,c'} \theta_{l:c,l':c'} x_{l:c} x_{l':c'}$$

where $l$ indexes positions in the sequence, $c$ indexes characters from the alphabet, and $x_{l:c}$ is an indicator variable equal to 1 when $c$ occurs at position $l$, and equal to 0 otherwise. The parameters of this model $\theta_0, \theta_{l:c}, \theta_{l:c,l':c'}$ are to be learned from data. 

`regression_type='GE'`: This indicates that we want to perform global epistasis (GE) regression. The key assumption of GE regression is that each measurement $y$ is a noisy readout of a predicte mesurement $\hat{y}$, which itself is a nonlinear function $g$ of the latent phenotype $\phi$. Thus,

$$ p(y | \phi) = p(y|\hat{y})~~\mathrm{where}~~\hat{y} = g(\phi).$$

The nonlinear function $g(\phi)$ is to be learned from data using a two-layer neural network having a default value of `ge_nonlinearity_hidden_nodes=50` hidden nodes. 

`ge_noise_model_type='Gaussian'`: This indicates that we wish for our noise model, $p(y|\hat{y})$, to be a Gaussian distribution. 

`ge_heteroskedasticitiy_order=2`: This indicates that we are using a heteroskedastic model. Specifically, we model the standard deviation $s$ of $p(y|\hat{y})$  an expoentiated quadratic polynomial in $\hat{y}$,

$$ s(\hat{y}) = \exp \left[ a_0 + a_1 \hat{y} + a_2 \hat{y}^2 \right], $$

where the coefficients $a_0, a_1, a_2$ are to be learned from data. 

To fit this model to training data, we do the following.

In [ ]:
# # Fit model to training data. Takes ~15 seconds
history = model.fit(epochs=20, learning_rate=.0005)

The variable `history` is the standard history object returned by TensorFlow. It contains the training loss and validation loss values as a function of training epoch, which can be visualized as follows:

In [ ]:
# Extract training loss and validation loss
loss_training = history.history['loss']
loss_validation = history.history['val_loss']

# Plot training history
fig, ax = plt.subplots(1,1)
ax.plot(loss_training, color='C2', label='training')
ax.plot(loss_validation, color='C3', label='validation')
ax.set_ylabel('loss')
ax.set_xlabel('epoch')
ax.set_title(f"training history")
ax.legend()

To assess model performance, we compute the correlation between $\hat{y}$ and $y$ on training data. To get values for $\hat{y}$, we use the method `model.x_to_yhat()`, which returns a numpy array the same shape as its input.

In [ ]:
# Predict measurement values (yhat) from sequence (x)
yhat_test = model.x_to_yhat(x_test)
print(f'yhat_test: {repr(yhat_test)}')

Next we plot $y$ against $\hat{y}$ and compute the corresponding $R^2$ value.

In [ ]:
# Compute R^2 between yhat and y_test
Rsq = np.corrcoef(yhat_test.ravel(), y_test)[0, 1]**2

# Plot y_test vs. yhat_test
fig, ax = plt.subplots(1,1)
ax.scatter(yhat_test, y_test, color='C0', s=5, alpha=.2, label='test data')
ax.set_xlabel('model prediction ($\hat{y}$)')
ax.set_ylabel('measurement ($y$)')
ax.set_title(f'performance ($R^2$={Rsq:.3})')

Note that instead of using `model.x_to_yhat()`, we can alternatively use `model.x_to_phi()` to first predict latent phenotype values $\phi$ from input sequences $x$, then use these $\phi$ to compute $\hat{y}$ values via `model.phi_to_yhat()`:

In [ ]:
# Predict latent phenotype values (phi) from sequence (x)
phi_test = model.x_to_phi(x_test)
print(f'phi_test: {repr(phi_test)}')

# Predict measurement values (yhat) from latent phenotype (phi)
yhat_test = model.phi_to_yhat(phi_test)
print(f'yhat_test: {repr(yhat_test)}')

It is worth plotting measurements $y$ against the predicted latent phenotypes $\phi$. On the same plot we can show the nonlineairty $g(\phi)$ by creating a grid of $\phi$ values and mapping these to $\hat{y}$ values using `model.phi_to_yhat()`. When doing this, it often makes sense to further plot confidence intervals of $p(y|\hat{y})$ using the function `model.yhat_to_yq()`.

In [ ]:
# Create a grid of phi values for plotting the GE nonlinearity
phi_lim = [min(phi_test)-.5, max(phi_test)+.5]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Compute yhat for each phi gridpoint
yhat_grid = model.phi_to_yhat(phi_grid)

# Compute 68% confidence interval in y for each value of yhat
yqs_grid = model.yhat_to_yq(yhat_grid, q=[0.16,0.84])

# Create figure and axes
fig, ax = plt.subplots(1, 1)
ax.scatter(phi_test, y_test, color='C0', s=5, alpha=.2, label='test data')
ax.plot(phi_grid, yhat_grid, linewidth=2, color='C1',
        label='$\hat{y} = g(\phi)$')
ax.plot(phi_grid, yqs_grid[:,0], linestyle='--', color='C1', label='68% CI')
ax.plot(phi_grid, yqs_grid[:,1], linestyle='--', color='C1')
ax.set_xlim(phi_lim)
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_ylabel('measurement ($y$)')
ax.set_title('measurement process')
ax.legend()

Strong sigmoidal nonlinearities like the one we find here are very common on MAVE experiments and it is critically important that one account for them when trying to model G-P maps. Note also that, in this case, we find that the standard deviation of $y$ about $\hat{y}$ decreases at larger $\hat{y}$ values. This illustrates the utility of infering a heteroskedastic noise model. 

Ultimately we want to view the parameters of the G-P map. MAVE-NN provides two methods to support such visualization, `mavenn.heatmap()` and `mavenn.pairwise_heatmap()`. First we retrieve the additive parameters $\theta_{l:c}$ in dataframe format using `get_additive_parameters()`:

In [ ]:
# Get values of additive parameters
additive_df = model.get_gpmap_parameters(which="additive")
additive_df.head()

Next we draw these using `mavenn.heatmap()`. Since all assayed sequences were of the form NNNGYNNNN, we additionally mask the parameters for 'A','C','U' at position 3 and for 'A','G' at position 4. 

In [ ]:
# Illustrate additive parameters as a heatmap
fig, ax = plt.subplots(1,1)
mask_dict = {3:'ACU',4:'AG'}
ax, cb = mavenn.heatmap(df=additive_df,
                        mask_dict=mask_dict, 
                        ax=ax)

Note that `mask_dict` can also be determined computationally from the training sequences using `model.get_mask_dict()`. 

In [ ]:
# Compute mask_dict from trainig data
mask_dict = mavenn.get_mask_dict(x_train, alphabet='rna')
mask_dict

Similarly, we retrieve the additive parameters $\theta_{l:c,l':c'}$ in dataframe format using `get_pairwise_parameters()`:

In [ ]:
# Get values of all model parameters, including pairwise parameters
pairwise_df = model.get_gpmap_parameters(which="pairwise")
pairwise_df.head()

We then draw thse using `mavenn.pairwise_heatmap()`, again with the specified mask in place. 

In [ ]:
# Illustrate pairwise parameters
fig, ax = plt.subplots(1,1, figsize=[9,5])
ax, cb = mavenn.heatmap_pairwise(pairwise_df, 
                                 mask_dict=mask_dict, 
                                 ax=ax)

An alternative way of interrogating model parameters is to compute the effects of mutations from a specific wild-type sequence of interest. In this case we will use the consensus splice site `"CAGGUAAGU"`.

In [ ]:
cons_ss = "CAGGUAAGU"

effects_df = mavenn.get_1pt_effects(model.x_to_phi, wt_seq=cons_ss, alphabet="rna")
effects_df.head()

In [ ]:
ax, cb = mavenn.heatmap(df=effects_df,
                        seq=cons_ss,
                        seq_kwargs={'s':100, 'marker':'o', 'color':'k'},
                        l_col="l",
                        c_col="c_mut",
                        value_col="dphi",
                        missing_values=0,
                        clim=[-2.5,1],
                        ccenter=0,
                        mask_dict=mask_dict)
ax.set_title(f'consus sequence: {cons_ss}')

As expected, all mutations away from the consensus sequence reduce $\phi$, which quantifies splice site strength.

Similarly, the effects of pairwise mutations can be visualized.

In [ ]:
effects_df = mavenn.get_2pt_effects(model.x_to_phi, wt_seq=cons_ss, alphabet="rna")
effects_df.head()

In [ ]:
# Illustrate pairwise parameters
fig, ax = plt.subplots(1,1, figsize=[9,5])
ax, cb = mavenn.heatmap_pairwise(effects_df, 
                                 l1_col="l1",
                                 l2_col="l2",
                                 c1_col="c1_mut",
                                 c2_col="c2_mut",
                                 value_col="ddphi",
                                 mask_dict=mask_dict,
                                 seq=cons_ss,
                                 ccenter=0,
                                 seq_kwargs={'s':10, 'marker':'o', 'color':'k'},
                                 ax=ax)